In [1]:
# Installation des bibliothèques nécessaires
!pip install peft
!pip install evaluate

!pip install -U transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

# Importation des Bibliothèques

In [2]:
# Importation des bibliothèques essentielles pour le traitement des données et le machine learning
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import os



# Bibliothèque pour l'évaluation des modèles
import evaluate

2025-11-19 20:33:44.709490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763584424.903890      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763584424.961037      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Chargement des données

In [3]:

# 📁 Dossier du dataset IMDB sur Kaggle
base = "/kaggle/input/imdb-dataset-of-50k-movie-reviews"

# 📄 Fichier principal
data_path = os.path.join(base, "IMDB Dataset.csv")

# ✅ Vérification
print("📂 Contenu du dossier :", os.listdir(base))
print("📄 Chargement du fichier :", data_path)

# 🔹 Chargement du dataset
df = pd.read_csv(data_path)

print("\n✅ Données chargées avec succès !")
print("📊 Dimensions :", df.shape)
print("📈 Colonnes :", list(df.columns))
print("\nAperçu :")
print(df.head())


📂 Contenu du dossier : ['IMDB Dataset.csv']
📄 Chargement du fichier : /kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv

✅ Données chargées avec succès !
📊 Dimensions : (50000, 2)
📈 Colonnes : ['review', 'sentiment']

Aperçu :
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
# Conversion des étiquettes en format numérique
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df = df.drop(['sentiment'], axis=1) 
df.head()

,review,label
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
# Séparer les données en ensembles d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Sélectionner un sous-ensemble aléatoire de N échantillons pour l'entraînement et le test
N = 1000
train_subset = train_df.sample(n=N, random_state=42)
test_subset = test_df.sample(n=N, random_state=42)

In [7]:
# Créer les datasets Hugging Face
train_dataset = Dataset.from_pandas(train_subset)
test_dataset = Dataset.from_pandas(test_subset)

In [8]:
train_dataset

Dataset({
    features: ['review', 'label', '__index_level_0__'],
    num_rows: 1000
})

In [9]:
test_dataset

Dataset({
    features: ['review', 'label', '__index_level_0__'],
    num_rows: 1000
})

# Finetuning complet

In [10]:
import torch
# Importe PyTorch, une bibliothèque pour les réseaux de neurones profonds et l'apprentissage automatique.

import pandas as pd
# Importe pandas, une bibliothèque pour la manipulation et l'analyse de données, souvent utilisée pour manipuler des tableaux de données.

from datasets import Dataset, DatasetDict
# Importe 'Dataset' et 'DatasetDict' de la bibliothèque 'datasets' de Hugging Face, utilisés pour gérer et préparer des ensembles de données pour l'entraînement de modèles.

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
# Importe plusieurs classes de la bibliothèque 'transformers' de Hugging Face:
# - DistilBertTokenizer : pour convertir du texte en tokens compréhensibles par DistilBERT.
# - DistilBertForSequenceClassification : une version de DistilBERT préparée pour la classification de séquences (par exemple, classification de texte).
# - TrainingArguments : pour configurer les paramètres d'entraînement.
# - Trainer : pour orchestrer le processus d'entraînement du modèle.

import torch.optim as optim
# Importe le sous-module 'optim' de PyTorch, qui contient divers algorithmes d'optimisation utilisés pour mettre à jour les poids du réseau pendant l'entraînement, tels que SGD, Adam, etc.


In [11]:
# Paramètres d'entraînement
model_checkpoint = "distilbert-base-uncased"  # Modèle distilbert pré-entraîné
lr = 2e-5  # Taux d'apprentissage
batch_size = 8  # Taille du lot
num_epochs = 3  # Nombre d'époques

In [12]:
# Charger le tokenizer pré-entraîné
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Convertir les colonnes en listes Python (simple)
train_texts = [str(x) for x in train_dataset["review"]]
test_texts = [str(x) for x in test_dataset["review"]]

# Tokenization correcte
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [13]:
# Étape 2 : Création des datasets PyTorch

train_labels = train_subset["label"].tolist()

train_datasetb = Dataset.from_dict({
    # 'input_ids' : Contient les identifiants de tokens obtenus après le processus de tokenisation du texte. 
    # Ces identifiants sont nécessaires pour que le modèle comprenne et traite le texte.
    'input_ids': train_encodings['input_ids'],
    # 'attention_mask' : Un masque d'attention qui indique au modèle quels tokens dans 'input_ids' sont significatifs 
    # (c'est-à-dire non-padding) et doivent être pris en compte lors du traitement.
    'attention_mask': train_encodings['attention_mask'],
    # 'labels' : Les étiquettes associées à chaque exemple d'entraînement. Ces étiquettes sont utilisées pour l'apprentissage supervisé,
    # où le modèle apprend à associer les entrées à ces étiquettes correctes.
    # 'torch.tensor()' est utilisé pour convertir la liste des étiquettes en tensor PyTorch, 
    # ce qui est nécessaire pour le traitement avec PyTorch.
    'labels': torch.tensor(train_labels)
})


test_labels = test_subset["label"].tolist()

test_datasetb = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(test_labels)
})

# Création d'un DatasetDict
datasetb = DatasetDict({
    'train': train_datasetb,
    'test': test_datasetb
})

In [14]:
# Création d'un modèle de classification de séquences basé sur DistilBERT
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Affichage du nombre de paramètres du modèle
num_params = sum(p.numel() for p in model.parameters())
print(f"Nombre de paramètres du modèle : {num_params}")

# Afficher l'architecture du modèle BERT
print(model)

Nombre de paramètres du modèle : 66955010
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
           

In [16]:
# Étape 4 : Définition des paramètres d'entraînement
# Configuration de l'optimiseur pour l'entraînement du modèle
optimizer = optim.AdamW(model.parameters(), lr=lr)
# Définition de la fonction de perte pour l'entraînement du modèle
loss_fn = torch.nn.CrossEntropyLoss()

In [17]:
# Importer la métrique pour l'évaluation
accuracy = evaluate.load("accuracy")

In [18]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy_value}

In [19]:
# Définir la correspondance entre les identifiants de classe et les étiquettes
id2label = {0: "Négatif", 1: "Positif"}
label2id = {"Négatif": 0, "Positif": 1}

In [20]:
liste_texte = [
    "This movie was a masterpiece.",
    "I was blown away by the acting.",
    "It's a classic that everyone should watch.",
    "The plot was confusing and hard to follow.",
    "The special effects were top-notch.",
    "I couldn't stop laughing throughout the movie.",
    "The soundtrack was incredible.",
    "It's a total waste of time.",
    "I'm still thinking about that ending.",
    "I wouldn't recommend it to anyone."
]


print("Prédictions du modèle non entraîné:")
print("-------------------------------------")
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt")
    # Calculer les logits
    logits = model(inputs).logits
    # Convertir les logits en étiquette
    predictions = torch.argmax(logits)

    print(texte + " - " + id2label[predictions.tolist()])

Prédictions du modèle non entraîné:
-------------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Positif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Positif
I'm still thinking about that ending. - Positif
I wouldn't recommend it to anyone. - Positif


In [27]:
# Étape 5 : Définition des arguments d'entraînement avec Transformers

training_args = TrainingArguments(
    output_dir=model_checkpoint + "-fullfinetuned-text-classification",# Le répertoire où les résultats de l'entraînement seront sauvegardés
    learning_rate=lr,# Taux d'apprentissage pour l'optimisation du modèle
    per_device_train_batch_size=batch_size,# Taille du lot d'entraînement par périphérique (GPU ou CPU)
    per_device_eval_batch_size=batch_size,# Taille du lot d'évaluation par périphérique (GPU ou CPU)
    num_train_epochs=num_epochs, # Nombre d'époques d'entraînement (combien de fois le modèle parcourt l'ensemble de données)
    weight_decay=0.01, # Terme de régularisation pour contrôler le poids de la pénalisation dans la fonction de perte

    eval_strategy="epoch",   # ⬅️ nouveau paramètre # Stratégie d'évaluation, ici "epoch" signifie évaluer à la fin de chaque époque
    save_strategy="epoch",   # ⬅️ inchangé # Stratégie de sauvegarde du modèle, ici "epoch" signifie sauvegarder le modèle à la fin de chaque époque
    load_best_model_at_end = True, # Charger le meilleur modèle à la fin de l'entraînement
    report_to="none"
)


In [28]:
# Étape 6 : Entraînement du modèle avec Trainer de Transformers
trainerfull = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasetb['train'],
    eval_dataset=datasetb['test'],
    compute_metrics=compute_metrics  # Utiliser la fonction de calcul des métriques

)

In [29]:
# Entraînement du modèle
trainerfull.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.621213,0.877000
2,No log,0.507758,0.898000
3,No log,0.530897,0.891000


TrainOutput(global_step=375, training_loss=0.12051556396484375, metrics={'train_runtime': 138.9141, 'train_samples_per_second': 21.596, 'train_steps_per_second': 2.7, 'total_flos': 397402195968000.0, 'train_loss': 0.12051556396484375, 'epoch': 3.0})

In [30]:
# Sauvegarde du modèle
trainerfull.save_model("trainer_full")

In [32]:
import torch

# 1. Définir le bon périphérique (device)
# Sur Kaggle avec GPU, ce sera "cuda". Sinon "cpu".
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Le modèle sera exécuté sur : {device}")

# 2. Sauvegarde du modèle (inchangé)
trainerfull.save_model("trainer_full")

# 3. Passer le modèle sur le bon périphérique (cuda)
model.to(device)

# 4. Afficher les prédictions
print("Prédictions du modèle entraîné :")
print("---------------------------------")

# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte ET l'envoyer sur le même périphérique que le modèle
    inputs = tokenizer.encode(texte, return_tensors="pt").to(device) # <--- Ici, on utilise 'device' (cuda)

    # Obtenir les logits
    with torch.no_grad(): # Optionnel mais recommandé pour l'inférence (économise la mémoire)
        logits = model(inputs).logits

    # Prédire la classe
    predictions = torch.max(logits, 1).indices

    # Afficher le résultat
    print(texte + " - " + id2label[predictions.tolist()[0]])

Le modèle sera exécuté sur : cuda
Prédictions du modèle entraîné :
---------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Négatif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Négatif
I'm still thinking about that ending. - Négatif
I wouldn't recommend it to anyone. - Négatif


# Layer freezing finetuning

In [34]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# --- CORRECTION ICI ---
# On extrait explicitement les textes et on force la conversion en string (str)
# pour éviter les erreurs si une ligne est vide ou mal formatée.
train_texts = [str(x) for x in train_dataset['review']]
test_texts = [str(x) for x in test_dataset['review']]

# Maintenant, on passe ces listes propres au tokenizer
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

In [35]:
train_datasetb = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(train_dataset['label'])
})

test_datasetb = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(test_dataset['label'])
})

In [36]:
# Modèle DistilBERT pour la classification de séquences
modelf = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
# Geler certaines couches du modèle
for param in modelf.base_model.parameters():
    param.requires_grad = False

In [40]:
from transformers import TrainingArguments

# Définir les arguments d'entraînement pour le Trainer de Hugging Face
training_args = TrainingArguments(
    # Répertoire où les résultats d'entraînement (modèles, configurations, etc.) seront sauvegardés.
    output_dir="./freezdistilbert_finetuned",

    # Nombre d'exemples par lot (batch) pour l'entraînement, défini pour chaque périphérique (GPU/CPU).
    per_device_train_batch_size=8,
    # Nombre d'exemples par lot pour l'évaluation, défini pour chaque périphérique.
    per_device_eval_batch_size=8,

    # --- CORRECTION ICI ---
    # "evaluation_strategy" est devenu "eval_strategy"
    # Stratégie d'évaluation à utiliser. Ici, "steps" signifie que l'évaluation aura lieu à intervalles réguliers de pas d'entraînement.
    eval_strategy="steps",

    # Fréquence d'évaluation et de sauvegarde
    # Nombre de pas d'entraînement à effectuer avant de réaliser une évaluation.
    eval_steps=100,
    # Nombre de pas d'entraînement après lesquels le modèle sera sauvegardé.
    save_steps=100,

    # Nombre d'époques
    # Nombre total d'époques d'entraînement à réaliser.
    num_train_epochs=3,
    # Taux d'apprentissage initial à utiliser pour l'optimiseur
    learning_rate=2e-5,
    # Limite du nombre total de sauvegardes de checkpoints à conserver. 
    # Ici, seul le checkpoint le plus récent sera conservé
    save_total_limit=1,
    
    # Pour éviter l'erreur wandb rencontrée précédemment
    report_to="none"
)

In [41]:
# Importer la métrique d'exactitude pour l'évaluation

accuracy = evaluate.load("accuracy")

In [42]:
# Fonction pour calculer l'exactitude
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    # Calcul de l'exactitude
    accuracy = np.mean(predictions == labels)

    return {"accuracy": accuracy}

In [43]:
from transformers import DataCollatorWithPadding  # Assurez-vous d'avoir cette ligne d'importation


# Créer un Trainer
trainerfreezing = Trainer(
    model=modelf,  # Utilisez le modèle DistilBERT que vous avez défini précédemment
    args=training_args,  # Les paramètres d'entraînement définis précédemment
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # Gère le padding des données lors de l'entraînement
    compute_metrics=compute_metrics,  # Fonction pour calculer les métriques d'évaluation
    train_dataset=train_datasetb,  # L'ensemble de données d'entraînement
    eval_dataset=test_datasetb  # L'ensemble de données d'évaluation
)

In [44]:
# Entraîner le modèle
trainerfreezing.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,0.679781,0.536000
200,No log,0.663598,0.774000
300,No log,0.656881,0.763000


TrainOutput(global_step=375, training_loss=0.6693230387369792, metrics={'train_runtime': 52.9813, 'train_samples_per_second': 56.624, 'train_steps_per_second': 7.078, 'total_flos': 397402195968000.0, 'train_loss': 0.6693230387369792, 'epoch': 3.0})

In [46]:
import torch

# 1. Définir le bon périphérique (device)
# Sur Kaggle avec GPU, ce sera "cuda". Sinon "cpu".
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Le modèle sera exécuté sur : {device}")

# 2. Sauvegarde du modèle (inchangé)
trainerfull.save_model("trainer_full")

# 3. Passer le modèle sur le bon périphérique (cuda)
model.to(device)

# 4. Afficher les prédictions
print("Prédictions du modèle entraîné :")
print("---------------------------------")

# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte ET l'envoyer sur le même périphérique que le modèle
    inputs = tokenizer.encode(texte, return_tensors="pt").to(device) # <--- Ici, on utilise 'device' (cuda)

    # Obtenir les logits
    with torch.no_grad(): # Optionnel mais recommandé pour l'inférence (économise la mémoire)
        logits = model(inputs).logits

    # Prédire la classe
    predictions = torch.max(logits, 1).indices

    # Afficher le résultat
    print(texte + " - " + id2label[predictions.tolist()[0]])

Le modèle sera exécuté sur : cuda
Prédictions du modèle entraîné :
---------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Négatif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Négatif
I'm still thinking about that ending. - Négatif
I wouldn't recommend it to anyone. - Négatif


# Finetuning LoRA

In [47]:
# Importation des outils spécifiques aux modèles transformers
from transformers import (
    AutoTokenizer,  # Pour le tokenizing des textes
    AutoModelForSequenceClassification,  # Pour charger un modèle pré-entraîné pour la classification de séquences
    TrainingArguments,  # Pour définir les paramètres d'entraînement
    Trainer,  # Pour entraîner le modèle
    DataCollatorWithPadding  # Pour gérer le padding des séquences lors de l'entraînement
)
# Importation des bibliothèques pour le fine-tuning avec PEFT (Parameter-Efficient Fine-tuning)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [48]:
# Définir le modèle que vous souhaitez utiliser pour la classification de séquences. Vous pouvez choisir un modèle pré-entraîné, comme 'distilbert-base-uncased' ou 'roberta-base'.

model_checkpoint = 'distilbert-base-uncased'

# Générer un modèle de classification à partir du modèle pré-entraîné
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


# Preprocessing

In [50]:
# Créer un tokenizer pour le modèle
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
tokenizer.pad_token
# Ajouter un jeton de padding s'il n'existe pas
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [51]:
# Créer une fonction pour tokenizer les données
def tokenize_function(examples):
    text = examples["review"]
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        padding=True,  # Ajout du padding directement ici
        truncation=True,
        max_length=512
    )
    return tokenized_inputs

In [52]:
# Tokenizer les ensembles d'entraînement et de test
# Créer un DatasetDict
# Tokenizer les ensembles d'entraînement et de test et créer un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['review', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [53]:
# Créer un DataCollator pour gérer le padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation du modèle

In [54]:
# Configuration du modèle LoRA (Low-Rank Adaptation) pour le fine-tuning

# Type de tâche, ici "SEQ_CLS" signifie classification de séquences
peft_config = LoraConfig(
    task_type="SEQ_CLS",  # Type de tâche du modèle

    # Facteur de régularisation "r" pour LoRA
    r=4,

    # Paramètre d'alpha pour LoRA, contrôlant l'importance de la régularisation LoRA
    lora_alpha=32,

    # Taux de dropout pour LoRA, qui détermine la probabilité de désactivation aléatoire des connexions
    lora_dropout=0.01,

    # Modules cibles pour l'application de la régularisation LoRA, ici uniquement le module 'q_lin' est ciblé
    target_modules=['q_lin']
)

In [55]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [56]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [57]:
# Hyperparamètres

# Taux d'apprentissage (learning rate) - C'est la vitesse à laquelle le modèle apprend
lr = 1e-3

# Taille du lot (batch size) - Le nombre d'exemples de données utilisés pour chaque mise à jour de poids du modèle
batch_size = 4

# Nombre d'époques (epochs) - Le nombre de fois que le modèle parcourt l'ensemble de données complet lors de l'entraînement
num_epochs = 3

In [59]:
# Définition des arguments d'entraînement

# Répertoire de sortie où les résultats de l'entraînement seront sauvegardés
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",  # Chemin du répertoire de sortie

    # Taux d'apprentissage
    learning_rate=lr,

    # Taille du lot
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Nombre d'époques
    num_train_epochs=num_epochs,

    # Terme de régularisation
    weight_decay=0.01,

    # --- CORRECTION ICI ---
    # Stratégie d'évaluation (nouveau nom)
    eval_strategy="epoch",

    # Stratégie de sauvegarde
    save_strategy="epoch",

    # Charger le meilleur modèle
    load_best_model_at_end=True,
    
    # Et toujours pour éviter le crash wandb
    report_to="none"
)

In [60]:
# Création de l'objet "trainer" pour entraîner le modèle

# Le modèle à entraîner
trainer = Trainer(
    model=model,  # Le modèle que vous avez configuré précédemment

    # Les arguments d'entraînement définis précédemment
    args=training_args,

    # L'ensemble de données d'entraînement tokenisé
    train_dataset=tokenized_dataset["train"],

    # L'ensemble de données de validation tokenisé
    eval_dataset=tokenized_dataset["test"],

    # Le tokenizer utilisé pour le prétraitement
    tokenizer=tokenizer,

    # Le "data_collator" qui gère le padding dynamique des exemples dans chaque lot pour qu'ils aient la même longueur
    data_collator=data_collator,

    # La fonction de calcul des métriques pour évaluer le modèle
    compute_metrics=compute_metrics
)

/tmp/ipykernel_48/2241369498.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [61]:
# Entraînement du modèle
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.540867,0.885000
2,0.329300,0.586220,0.875000
3,0.329300,0.648329,0.883000


TrainOutput(global_step=750, training_loss=0.2689953816731771, metrics={'train_runtime': 154.9282, 'train_samples_per_second': 19.364, 'train_steps_per_second': 4.841, 'total_flos': 403199004672000.0, 'train_loss': 0.2689953816731771, 'epoch': 3.0})

# Generate prediction

In [63]:
import torch

# 1. Définir le bon périphérique (device)
# Sur Kaggle avec GPU, ce sera "cuda". Sinon "cpu".
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Le modèle sera exécuté sur : {device}")

# 2. Sauvegarde du modèle (inchangé)
trainerfull.save_model("trainer_full")

# 3. Passer le modèle sur le bon périphérique (cuda)
model.to(device)

# 4. Afficher les prédictions
print("Prédictions du modèle entraîné :")
print("---------------------------------")

# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte ET l'envoyer sur le même périphérique que le modèle
    inputs = tokenizer.encode(texte, return_tensors="pt").to(device) # <--- Ici, on utilise 'device' (cuda)

    # Obtenir les logits
    with torch.no_grad(): # Optionnel mais recommandé pour l'inférence (économise la mémoire)
        logits = model(inputs).logits

    # Prédire la classe
    predictions = torch.max(logits, 1).indices

    # Afficher le résultat
    print(texte + " - " + id2label[predictions.tolist()[0]])

Le modèle sera exécuté sur : cuda
Prédictions du modèle entraîné :
---------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Négatif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Négatif
I'm still thinking about that ending. - Négatif
I wouldn't recommend it to anyone. - Négatif
